[카카오 아레나 포럼
_CF](https://arena.kakao.com/forum/topics/227)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install fire

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=d23f5b7b8b1934587c6afff2d75c58f49dc002743f9e46efcae29035d3364ccc
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [ ]:
import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle
from scipy.sparse import hstack

#song_meta = pd.read_json("drive/My Drive/KAKAO/song_meta.json")

In [ ]:
cd /content/drive/My Drive/Kakao arena

/content/drive/My Drive/Kakao arena


# Pre_processing
feature 추가 및 song_cnt >= 5

In [ ]:
train = pd.read_json('data/meta/train.json', typ = 'frame')
song_meta = pd.read_json('/content/drive/My Drive/Kakao arena/data/meta/song_meta.json', typ = 'frame',encoding='utf-8')
plylst_meta = pd.DataFrame(train[['id','tags','plylst_title']])

In [ ]:
plylst_song_map = train[['id', 'songs']]

plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

In [ ]:
plylst_song_map = plylst_song_map.astype(float)
plylst_song_map = plylst_song_map.astype(int)

In [ ]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [ ]:
plylst_song_map = pd.merge(plylst_song_map,song_meta,how='left',left_on='songs',right_index=True)
plylst_song_map = plylst_song_map.drop('id_y',axis=1)
plylst_song_map.rename(columns={'id_x':'id'},inplace=True)
plylst_song_map.head()

,id,songs,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
0,61281,525514,"[GN1402, GN1401]",20130506,Hey Little Girl,2200223,[734201],Hey Little Girl,[GN1400],[The Sol]
1,61281,129701,"[GN0901, GN0902, GN1001]",20130917,Brass,2201802,[536907],Octagon,"[GN0900, GN1000]",[Royal Bangs]
2,61281,383374,"[GN1012, GN1005, GN1001]",19911021,Monsters Under The Bed,2216938,[166978],The Road,[GN1000],[Honeymoon Suite]
3,61281,562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,United,43227,[19035],Honeymoon,"[GN0900, GN1000]",[Phoenix]
4,61281,297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,Back To Bedlam,303657,[170117],High,"[GN0900, GN1000]",[James Blunt]


In [ ]:
plylst_meta = pd.DataFrame(train[['id','tags','plylst_title']])
plylst_meta

,id,tags,plylst_title
0,61281,[락],여행같은 음악
1,10532,"[추억, 회상]",요즘 너 말야
2,76951,"[까페, 잔잔한]","편하게, 잔잔하게 들을 수 있는 곡.-"
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",크리스마스 분위기에 흠뻑 취하고 싶을때
4,27616,[댄스],추억의 노래 ㅋ
...,...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",METAL E'SM #2
115067,106976,[일렉],빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",#1. 눈물이 앞을 가리는 나의_이야기
115069,131982,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP


In [ ]:
for column in plylst_song_map.columns[1:]:
  plylst_sum = pd.DataFrame(plylst_song_map.groupby('id')[column].apply(list))
  plylst_sum = plylst_sum.reset_index()

  plylst_sum['id'] = plylst_sum['id'].astype(str).astype(int)
  plylst_meta = pd.merge(plylst_meta,plylst_sum,left_on='id',right_on='id',how='inner')

plylst_meta

,id,tags,plylst_title,songs,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
0,61281,[락],여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[[GN1402, GN1401], [GN0901, GN0902, GN1001], [...","[20130506, 20130917, 19911021, 20000919, 20050...","[Hey Little Girl, Brass, Monsters Under The Be...","[2200223, 2201802, 2216938, 43227, 303657, 360...","[[734201], [536907], [166978], [19035], [17011...","[Hey Little Girl, Octagon, The Road, Honeymoon...","[[GN1400], [GN0900, GN1000], [GN1000], [GN0900...","[[The Sol], [Royal Bangs], [Honeymoon Suite], ..."
1,10532,"[추억, 회상]",요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[[GN0101, GN0103], [GN0601, GN0605], [GN0104, ...","[20030314, 20061204, 20061204, 20071023, 20080...","[His Ballad Ⅰ, Audition, Audition, 혜성, 바보에게......","[30473, 333687, 333687, 359890, 375797, 390452...","[[26], [203912], [203912], [203912], [4263], [...","[한사람을 위한 마음, Audition (Time2Rock), 기다리다, 좀 더 둘...","[[GN0100], [GN0600], [GN0100], [GN0600], [GN01...","[[이승환], [윤하 (YOUNHA)], [윤하 (YOUNHA)], [윤하 (YOU..."
2,76951,"[까페, 잔잔한]","편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[[GN0401, GN0403], [GN0401, GN0403], [GN0501, ...","[20081029, 20081029, 20140818, 20141126, 20141...","[Soul Light, Soul Light, 너를 사랑하니까, 247, 그랬으면 (...","[464152, 464152, 2274784, 2292294, 2296308, 23...","[[262975], [262975], [782381], [165235], [7321...","[도시의 밤, I`m Alright, 너를 좋아하니까, 247 (Feat. AMJ)...","[[GN0400], [GN0400], [GN0500, GN0600], [GN0400...","[[소울라이츠], [소울라이츠], [뷰티핸섬], [소울사이어티], [카멜], [오디..."
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[[GN0908, GN1509, GN0901, GN2207, GN1501, GN15...","[20191115, 19981201, 20061124, 20101102, 20101...",[Frozen 2 (Original Motion Picture Soundtrack/...,"[10352535, 3167, 351629, 1074764, 1074764, 107...","[[232538, 838137], [925], [220998], [2274], [2...",[Into the Unknown (From &#34;Frozen 2&#34;/Sou...,"[[GN1500, GN0900, GN2200], [GN0100], [GN0900],...","[[Idina Menzel, Aurora], [김동률], [Ashley Tisdal..."
4,27616,[댄스],추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...","[[GN0101], [GN0101, GN0103], [GN2502, GN2506, ...","[20010619, 19970520, 19970514, 19940900, 20020...","[First Story, 2집- 타인, 학원별곡, Blue City, No. 1, ...","[6846, 5458, 5368, 3957, 16305, 3259, 3248, 32...","[[10575], [100045], [100052], [1130], [2165], ...","[눈물에 얼굴을 묻는다, 타인, 학원별곡 (學園別曲), 날 떠나지마, No.1, D...","[[GN0100], [GN0100], [GN2500, GN0200], [GN0200...","[[장나라], [영턱스클럽], [젝스키스], [박진영], [보아 (BoA)], [D..."
...,...,...,...,...,...,...,...,...,...,...,...,...
115066,120325,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...","[[GN1006, GN1013, GN1001], [GN1007, GN1013, GN...","[20040117, 20011108, 20011108, 20011108, 20001...","[Devil Driver, The War Of Art, The War Of Art,...","[33725, 65802, 65802, 65802, 316229, 349526, 3...","[[108340], [105339], [105339], [105339], [1011...","[Cry For Me Sky (Eulogy Of The Scorned), Song ...","[[GN1000], [GN1000], [GN1000], [GN1000], [GN10...","[[DevilDriver], [American Head Charge], [Ameri..."
115067,106976,[일렉],빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...","[[GN1104, GN1101], [GN1104, GN1102, GN1101], [...","[20151208, 20151216, 20150224, 20150820, 20150...","[Crank (Running Away), Make That Booty Bounce,...","[2655210, 2656912, 2305743, 2335671, 2639899, ...","[[839169], [858588], [687821], [775358], [7361...","[Crank (Running Away) (Original Mix), Make Tha...","[[GN1100], [GN1100], [GN1100], [GN1100], [GN11...","[[Komes], [Ctrl Alt Del], [Parachute Youth], [..."
115068,11343,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...","[[GN0105, GN0101], [GN0105, GN0101], [GN2502, ...","[20170216, 20190409, 20070608, 2008

In [ ]:
list_columns = ['song_gn_dtl_gnr_basket','artist_id_basket','song_gn_gnr_basket','artist_name_basket']

for column in list_columns:
  plylst_meta[f'{column}_flatten'] = plylst_meta[column].map(lambda x : sum(x,[])) # 이중리스트 단일 리스트로. (list_columns의 column들이 이중리스트인 것들)
  plylst_meta[f'{column}_unique'] = plylst_meta[f'{column}_flatten'].map(lambda x : list(set(x))) # 리스트 > 집합 > 리스트로 unique한 값 남김
  plylst_meta[f'{column}_count'] = plylst_meta[f'{column}_unique'].map(lambda x : len(x)) # unique한 것 개수 세기

plylst_meta

,id,tags,plylst_title,songs,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,song_gn_dtl_gnr_basket_flatten,song_gn_dtl_gnr_basket_unique,song_gn_dtl_gnr_basket_count,artist_id_basket_flatten,artist_id_basket_unique,artist_id_basket_count,song_gn_gnr_basket_flatten,song_gn_gnr_basket_unique,song_gn_gnr_basket_count,artist_name_basket_flatten,artist_name_basket_unique,artist_name_basket_count
0,61281,[락],여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[[GN1402, GN1401], [GN0901, GN0902, GN1001], [...","[20130506, 20130917, 19911021, 20000919, 20050...","[Hey Little Girl, Brass, Monsters Under The Be...","[2200223, 2201802, 2216938, 43227, 303657, 360...","[[734201], [536907], [166978], [19035], [17011...","[Hey Little Girl, Octagon, The Road, Honeymoon...","[[GN1400], [GN0900, GN1000], [GN1000], [GN0900...","[[The Sol], [Royal Bangs], [Honeymoon Suite], ...","[GN1402, GN1401, GN0901, GN0902, GN1001, GN101...","[GN1013, GN0902, GN1901, GN1012, GN1003, GN130...",18,"[734201, 536907, 166978, 19035, 170117, 19035,...","[166978, 182425, 495811, 170117, 227971, 57773...",17,"[GN1400, GN0900, GN1000, GN1000, GN0900, GN100...","[GN0900, GN1000, GN1100, GN1900, GN1400, GN1300]",6,"[The Sol, Royal Bangs, Honeymoon Suite, Phoeni...","[OneRepublic, Man Man, Bruno Mars, Cody Simpso...",17
1,10532,"[추억, 회상]",요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[[GN0101, GN0103], [GN0601, GN0605], [GN0104, ...","[20030314, 20061204, 20061204, 20071023, 20080...","[His Ballad Ⅰ, Audition, Audition, 혜성, 바보에게......","[30473, 333687, 333687, 359890, 375797, 390452...","[[26], [203912], [203912], [203912], [4263], [...","[한사람을 위한 마음, Audition (Time2Rock), 기다리다, 좀 더 둘...","[[GN0100], [GN0600], [GN0100], [GN0600], [GN01...","[[이승환], [윤하 (YOUNHA)], [윤하 (YOUNHA)], [윤하 (YOU...","[GN0101, GN0103, GN0601, GN0605, GN0104, GN010...","[GN0502, GN1601, GN1504, GN0801, GN0104, GN060...",29,"[26, 203912, 203912, 203912, 4263, 1868, 1121,...","[203912, 774540, 602511, 184730, 26, 925, 5371...",21,"[GN0100, GN0600, GN0100, GN0600, GN0100, GN060...","[GN1800, GN2600, GN0600, GN0500, GN0900, GN010...",11,"[이승환, 윤하 (YOUNHA), 윤하 (YOUNHA), 윤하 (YOUNHA), 박...","[이상순, 이승철, Manhattan Jazz Orchestra, 윤하 (YOUNH...",21
2,76951,"[까페, 잔잔한]","편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[[GN0401, GN0403], [GN0401, GN0403], [GN0501, ...","[20081029, 20081029, 20140818, 20141126, 20141...","[Soul Light, Soul Light, 너를 사랑하니까, 247, 그랬으면 (...","[464152, 464152, 2274784, 2292294, 2296308, 23...","[[262975], [262975], [782381], [165235], [7321...","[도시의 밤, I`m Alright, 너를 좋아하니까, 247 (Feat. AMJ)...","[[GN0400], [GN0400], [GN0500, GN0600], [GN0400...","[[소울라이츠], [소울라이츠], [뷰티핸섬], [소울사이어티], [카멜], [오디...","[GN0401, GN0403, GN0401, GN0403, GN0501, GN060...","[GN1703, GN0101, GN0402, GN1701, GN0502, GN030...",19,"[262975, 262975, 782381, 165235, 732108, 86902...","[205707, 542360, 757273, 869026, 782242, 77341...",23,"[GN0400, GN0400, GN0500, GN0600, GN0400, GN010...","[GN2600, GN0600, GN0500, GN0100, GN0400, GN080...",8,"[소울라이츠, 소울라이츠, 뷰티핸섬, 소울사이어티, 카멜, 오디오라인 (Audiol...","[카멜, 쏠라티, 소울사이어티, 프롬유 (From.U), 소심한 오빠들, 김새한길,...",23
3,147456,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...","[[GN0908, GN1509, GN0901, GN2207, GN1501, GN15...","[20191115, 19981201, 20061124, 20101102, 20101...",[Frozen 2 (Original Motion Picture Soundtrack/...,"[10352535, 3167, 351629, 1074764, 1074764, 107...","[[232538, 838137], [925], [220998], [2274], [2...",[Into the Unknown (From &#34;Frozen 2&#34;/Sou...,"[[GN1500, GN0900, GN2200], [GN0100], [GN0900],...","[[Idina Menzel, Aurora], [김동률], [Ashley Tisdal...","[GN0908, GN1509, GN0901, GN2207, GN1501, GN150...","[GN0502, GN0506, GN1504, GN0801, GN0708, GN060...",31,"[232538, 838137, 925, 220998, 2274, 2274, 2274...","[2640648, 773385, 1202067, 925, 101921, 205858...",4

In [ ]:
meta = plylst_meta[['id','tags','plylst_title','songs','issue_date','song_gn_gnr_basket_flatten','artist_id_basket_flatten','artist_id_basket_count','song_gn_gnr_basket_count']]

In [ ]:
meta.head(3)

,id,tags,plylst_title,songs,issue_date,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count
0,61281,[락],여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[20130506, 20130917, 19911021, 20000919, 20050...","[GN1400, GN0900, GN1000, GN1000, GN0900, GN100...","[734201, 536907, 166978, 19035, 170117, 19035,...",17,6
1,10532,"[추억, 회상]",요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[20030314, 20061204, 20061204, 20071023, 20080...","[GN0100, GN0600, GN0100, GN0600, GN0100, GN060...","[26, 203912, 203912, 203912, 4263, 1868, 1121,...",21,11
2,76951,"[까페, 잔잔한]","편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[20081029, 20081029, 20140818, 20141126, 20141...","[GN0400, GN0400, GN0500, GN0600, GN0400, GN010...","[262975, 262975, 782381, 165235, 732108, 86902...",23,8


In [ ]:
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

value = plylst_song_map['songs'].value_counts() > 5

df = plylst_song_map['songs'].value_counts().rename_axis('unique_values').to_frame('counts')
df = df.reset_index() ; df
over_5 = df[df['counts']>=5]

requred_song = over_5['unique_values'].tolist()
requred_song = set(list(map(int, requred_song)))

In [ ]:
df = pd.DataFrame(columns=['over5_songs'])
meta = pd.concat([meta,df])

for i in range(meta.shape[0]):     
    result = set(meta['songs'][i]) & requred_song
    meta['over5_songs'][i] = list(result)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
del meta['songs']
meta.rename(columns = {'over5_songs' : 'songs'}, inplace = True)
meta['updt_date'] = train['updt_date']
meta.head()

,id,tags,plylst_title,issue_date,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,songs,updt_date
0,61281.0,[락],여행같은 음악,"[20130506, 20130917, 19911021, 20000919, 20050...","[GN1400, GN0900, GN1000, GN1000, GN0900, GN100...","[734201, 536907, 166978, 19035, 170117, 19035,...",17.0,6.0,"[562083, 297861, 72552, 525514, 531057, 139541...",2013-12-19 18:36:19.000
1,10532.0,"[추억, 회상]",요즘 너 말야,"[20030314, 20061204, 20061204, 20071023, 20080...","[GN0100, GN0600, GN0100, GN0600, GN0100, GN060...","[26, 203912, 203912, 203912, 4263, 1868, 1121,...",21.0,11.0,"[414721, 461062, 442765, 532114, 6546, 645653,...",2014-12-02 16:19:42.000
2,76951.0,"[까페, 잔잔한]","편하게, 잔잔하게 들을 수 있는 곡.-","[20081029, 20081029, 20140818, 20141126, 20141...","[GN0400, GN0400, GN0500, GN0600, GN0400, GN010...","[262975, 262975, 782381, 165235, 732108, 86902...",23.0,8.0,"[348801, 556426, 32017, 454550, 354465, 203558...",2017-08-28 07:09:34.000
3,147456.0,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",크리스마스 분위기에 흠뻑 취하고 싶을때,"[20191115, 19981201, 20061124, 20101102, 20101...","[GN1500, GN0900, GN2200, GN0100, GN0900, GN130...","[232538, 838137, 925, 220998, 2274, 2274, 2274...",41.0,11.0,"[453762, 273672, 521739, 556301, 334095, 54273...",2019-12-05 15:15:18.000
4,27616.0,[댄스],추억의 노래 ㅋ,"[20010619, 19970520, 19970514, 19940900, 20020...","[GN0100, GN0100, GN2500, GN0200, GN0200, GN250...","[10575, 100045, 100052, 1130, 2165, 100005, 10...",37.0,6.0,"[503552, 617473, 553610, 5130, 224139, 569867,...",2011-10-25 13:54:56.000


In [ ]:
## 저장
meta.to_json('meta.json', orient='table')

In [570]:
meta = pd.read_json('meta.json',orient='table')

In [571]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(meta, test_size=0.2, shuffle=True, random_state=1003)

# Util function

In [572]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))

In [573]:
class CustomEvaluator:
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(101)]

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)] # self._idcgs[len(gt)] = 0

    def _eval(self, gt_fname, rec_fname):
        gt_playlists = load_json(gt_fname)
        gt_dict = {g["id"]: g for g in gt_playlists} # 정답 {플레이리스트 아이디 : 플레이리스트 정보 쭉}
        rec_playlists = load_json(rec_fname)

        gt_ids = set([g["id"] for g in gt_playlists]) # 정답 플레이리스트 아이디
        rec_ids = set([r["id"] for r in rec_playlists]) # 답안 플레이리스트 아이디 

        '''
        if gt_ids != rec_ids:
            print(f'gt_ids:{len(gt_ids)}')
            print(f'rec_ids:{len(rec_ids)}')
            raise Exception("결과의 플레이리스트 수가 올바르지 않습니다.")

        rec_song_counts = [len(p["songs"]) for p in rec_playlists] #답안 플레이리스트 song 개수 집합으로. (100 아닌거 있나 걸러내려고)
        rec_tag_counts = [len(p["tags"]) for p in rec_playlists]

        if set(rec_song_counts) != set([100]):
            print(f'rec_song_counts={set(rec_song_counts)}')
            raise Exception("추천 곡 결과의 개수가 맞지 않습니다.")

        if set(rec_tag_counts) != set([10]):
            raise Exception("추천 태그 결과의 개수가 맞지 않습니다.")

        rec_unique_song_counts = [len(set(p["songs"])) for p in rec_playlists]
        rec_unique_tag_counts = [len(set(p["tags"])) for p in rec_playlists]

        if set(rec_unique_song_counts) != set([100]):
            print(f'rec_unique_song_counts={set(rec_unique_song_counts)}')
            raise Exception("한 플레이리스트에 중복된 곡 추천은 허용되지 않습니다.")

        if set(rec_unique_tag_counts) != set([10]):
            print(f'rec_unique_tag_counts={set(rec_unique_tag_counts)}')
            raise Exception("한 플레이리스트에 중복된 태그 추천은 허용되지 않습니다.")
        '''

        music_ndcg = 0.0
        tag_ndcg = 0.0

        for rec in rec_playlists:
            gt = gt_dict[rec["id"]]
            music_ndcg += self._ndcg(gt["songs"], rec["songs"][:100])
            tag_ndcg += self._ndcg(gt["tags"], rec["tags"][:10])

        music_ndcg = music_ndcg / len(rec_playlists)
        tag_ndcg = tag_ndcg / len(rec_playlists)
        score = music_ndcg * 0.85 + tag_ndcg * 0.15

        return music_ndcg, tag_ndcg, score

    def evaluate(self, gt_fname, rec_fname): # gt > 정답, rec > 제출 답안
        try:
            music_ndcg, tag_ndcg, score = self._eval(gt_fname, rec_fname)
            print(f"Music nDCG: {music_ndcg:.6}")
            print(f"Tag nDCG: {tag_ndcg:.6}")
            print(f"Score: {score:.6}")
        except Exception as e:
            print(e)

# Make Target(score)

In [574]:
train['istrain'] = 1
test['istrain'] = 0

n_train = len(train)
n_test = len(test)

# train + test
plylst = pd.concat([train,test], ignore_index=True)
plylst["nid"] = range(n_train+n_test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,tags,plylst_title,issue_date,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,songs,updt_date,istrain,nid
0,104383.0,"[설레임, 설렘, 사랑]",설레임 한가득 노래모음,"[20180312, 20180720, 20020719, 20020801, 20000...","[GN2500, GN0100, GN0300, GN0100, GN0100, GN060...","[490981, 239366, 3305, 1582, 100391, 11538, 27...",55.0,10.0,"[317056, 464257, 656512, 610179, 348165, 58676...",2020-02-02 16:15:59.000,1,0
1,28024.0,"[우울, 새벽, 밤, 고독]",멜랑꼴리한 밤에 고독을 짓씹으며,"[20080612, 20081111, 20081111, 20120619, 20120...","[GN0100, GN0900, GN0900, GN1000, GN1000, GN030...","[3305, 258383, 258383, 106732, 425140, 108356,...",23.0,11.0,"[536838, 260107, 164235, 449683, 329113, 14363...",2018-01-23 20:21:27.000,1,1
2,120331.0,"[커피, 잔잔한, 카페, 발라드]",뜨거운 커피 한잔과 함께 듣는 노래,"[20180330, 20150701, 20150701, 20160510, 20161...","[GN0500, GN0100, GN0500, GN0600, GN0500, GN060...","[681456, 242988, 242988, 770167, 103486, 24298...",16.0,7.0,"[106017, 327522, 338461, 33829, 548005, 233641...",2018-04-18 17:39:48.000,1,2
3,109188.0,"[여름, 기분전환]",무더운날 청량감 넘치는 연주곡들,"[20120426, 20120504, 20130107, 20130429, 20140...","[GN1800, GN1800, GN1800, GN1800, GN1800, GN180...","[623915, 618273, 713816, 713738, 713743, 71373...",8.0,1.0,"[497090, 167683, 298053, 253320, 278824, 32945...",2015-09-03 16:54:26.000,1,3
4,65498.0,"[매장음악, 까페]",아침을 열어주는 분위기 있는 카페 매장 음악,"[20150707, 20150707, 20150707, 20150707, 20150...","[GN1600, GN1600, GN1600, GN1600, GN1600, GN160...","[873816, 873816, 873816, 873816, 873334, 87333...",2.0,1.0,"[436992, 577664, 72194, 440965, 535046, 379655...",2015-09-03 17:01:15.000,1,4


In [575]:
# id <-> nid
plylst_id_nid = dict(zip(plylst["id"],plylst["nid"]))
plylst_nid_id = dict(zip(plylst["nid"],plylst["id"]))

In [576]:
plylst_tag = plylst['tags'][0:3]
for tgs in plylst_tag:
  print(tgs)
  for tg in tgs:
    print(tg)

tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])

tag_dict = {x:tag_counter[x] for x in tag_counter}

['설레임', '설렘', '사랑']
설레임
설렘
사랑
['우울', '새벽', '밤', '고독']
우울
새벽
밤
고독
['커피', '잔잔한', '카페', '발라드']
커피
잔잔한
카페
발라드
Counter({'설레임': 1, '설렘': 1, '사랑': 1, '우울': 1, '새벽': 1, '밤': 1, '고독': 1, '커피': 1, '잔잔한': 1, '카페': 1, '발라드': 1})
*******************
{'설레임': 1, '설렘': 1, '사랑': 1, '우울': 1, '새벽': 1, '밤': 1, '고독': 1, '커피': 1, '잔잔한': 1, '카페': 1, '발라드': 1}


In [577]:
plylst_tag = plylst['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs]) # counter > 데이터 개수 셀 때 유용
tag_dict = {x:tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()

for i,t in enumerate(tag_dict):
  tag_id_tid[t] = i
  tag_tid_id[i] = t

n_tags = len(tag_dict)

In [578]:
plylst_song = plylst['songs']
song_counter = Counter([sg for sgs in plylst_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()

for i, t in enumerate(song_dict):
  song_id_sid[t] = i
  song_sid_id[i] = t

n_songs = len(song_dict)

In [579]:
# plylst의 songs와 tags를 새로운 id로 변환하여 데이터프레임에 추가
plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None]) # get ; key로 value 얻기
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

In [582]:
plylst_use = plylst[['id','istrain','nid','updt_date','songs_id','tags_id','plylst_title','song_gn_gnr_basket_flatten','artist_id_basket_flatten','artist_id_basket_count','song_gn_gnr_basket_count','tags']]
plylst_use.loc[:,'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [584]:
plylst_train = plylst_use.iloc[:n_train,:]
plylst_test = plylst_use.iloc[n_train:,:]

In [586]:
np.random.seed(33)
n_sample = 23015

test = plylst_test.iloc[np.random.choice(range(n_test), size=n_sample,replace=False),:]

In [589]:
import math
from tqdm import tqdm

def rating(number):
  return [-math.log(x+1,2) +8.66 for x in range(number)]

# csr_matrix > (행,열)로 데이터 위치 표시
row = np.repeat(range(n_train),plylst_train['num_songs'])
col = [song for songs in plylst_train['songs_id'] for song in songs]
dat_series = plylst_train['num_songs'].map(rating)
dat = [y for x in dat_series for y in x]
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs))

row = np.repeat(range(n_train), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat_series = plylst_train['num_tags'].map(rating)
dat = [y for x in dat_series for y in x]
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))

train_tags_A # 태그는 그대로

<92056x29160 sparse matrix of type '<class 'numpy.float64'>'
	with 380817 stored elements in Compressed Sparse Row format>

In [590]:
train_songs_A  # 14만 곡으로 줄었음

<92056x144674 sparse matrix of type '<class 'numpy.float64'>'
	with 3626357 stored elements in Compressed Sparse Row format>

In [591]:
target = train_songs_A.T

# Featrue engineering
input이 spare 하지 않다 -> 구지 FM을 쓸 필요가 없다.  
XGBoost나 다른 regression 고려

In [592]:
train = plylst_train

### Season 
봄, 여름, 가을, 겨울 

In [593]:
train['updt_date'] = pd.to_datetime(train['updt_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [594]:
train['date'] = train['updt_date'].dt.date         # YYYY-MM-DD(문자)
train['year']     = train['updt_date'].dt.year         # 연(4자리숫자)
train['month']      = train['updt_date'].dt.month        # 월(숫자)
train['season'] = train['updt_date'].dt.quarter

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [595]:
train['season'][train['month'].isin([1,2,12])] = 4  # 겨울
train['season'][train['month'].isin([3,4,5])] = 1   # 봄
train['season'][train['month'].isin([6,7,8])] = 2  # 여름
train['season'][train['month'].isin([9,10,11])] = 3  # 가을

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/

In [596]:
train.head(3)

,id,istrain,updt_date,songs_id,tags_id,plylst_title,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,tags,num_songs,num_tags,date,year,month,season
nid,,,,,,,,,,,,,,,,,
0,104383.0,1,2020-02-02 16:15:59,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2]",설레임 한가득 노래모음,"[GN2500, GN0100, GN0300, GN0100, GN0100, GN060...","[490981, 239366, 3305, 1582, 100391, 11538, 27...",55.0,10.0,"[설레임, 설렘, 사랑]",74,3,2020-02-02,2020,2,4
1,28024.0,1,2018-01-23 20:21:27,"[74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 8...","[3, 4, 5, 6]",멜랑꼴리한 밤에 고독을 짓씹으며,"[GN0100, GN0900, GN0900, GN1000, GN1000, GN030...","[3305, 258383, 258383, 106732, 425140, 108356,...",23.0,11.0,"[우울, 새벽, 밤, 고독]",26,4,2018-01-23,2018,1,4
2,120331.0,1,2018-04-18 17:39:48,"[100, 101, 102, 103, 104, 105, 106, 107, 108, ...","[7, 8, 9, 10]",뜨거운 커피 한잔과 함께 듣는 노래,"[GN0500, GN0100, GN0500, GN0600, GN0500, GN060...","[681456, 242988, 242988, 770167, 103486, 24298...",16.0,7.0,"[커피, 잔잔한, 카페, 발라드]",19,4,2018-04-18,2018,4,1


### Year_section

In [597]:
df = pd.DataFrame(columns=['year_section'])
train = pd.concat([train,df])

In [598]:
train['year_section'][train['year'].isin([2005,2006,2007,2008,2009,2010,2011,2012])] = 1
train['year_section'][train['year'].isin([2013,2014])] = 2
train['year_section'][train['year'].isin([2015,2016])] = 3
train['year_section'][train['year'].isin([2017,2018])] = 4
train['year_section'][train['year'].isin([2019,2020])] = 5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [599]:
del train['date']
del train['updt_date']
del train['istrain']

기존 month, year은 공선성 문제가 있을테니 제거?? -> ㅇㅇ

In [601]:
X = train.drop('songs_id',axis= 1)
X = X.drop('num_songs',axis=1)
X = X.drop('num_tags',axis=1)
X = X.drop('tags_id', axis=1)

In [602]:
X.head(3)
""" plylst_title => 타이틀 임배딩 벡터?
    song_gn_gnr_basket_flatten => 장르 cnt-hot-encoding
    artist_id_basket_flatten => 아티스트 임배딩 벡터?
    tags => 태그 임배딩?

    일단 빼고 implement
"""

,id,plylst_title,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,tags,year,month,season,year_section
0,104383.0,설레임 한가득 노래모음,"[GN2500, GN0100, GN0300, GN0100, GN0100, GN060...","[490981, 239366, 3305, 1582, 100391, 11538, 27...",55.0,10.0,"[설레임, 설렘, 사랑]",2020.0,2.0,4.0,5
1,28024.0,멜랑꼴리한 밤에 고독을 짓씹으며,"[GN0100, GN0900, GN0900, GN1000, GN1000, GN030...","[3305, 258383, 258383, 106732, 425140, 108356,...",23.0,11.0,"[우울, 새벽, 밤, 고독]",2018.0,1.0,4.0,4
2,120331.0,뜨거운 커피 한잔과 함께 듣는 노래,"[GN0500, GN0100, GN0500, GN0600, GN0500, GN060...","[681456, 242988, 242988, 770167, 103486, 24298...",16.0,7.0,"[커피, 잔잔한, 카페, 발라드]",2018.0,4.0,1.0,4


### 장르 임배딩

In [603]:
from collections import Counter
genre_gn_all = pd.read_json('/content/drive/My Drive/Kakao arena/data/meta/genre_gn_all.json', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']
code2idx = {code:i for i, code in gnr_code['gnr_code'].reset_index(drop=True).items()}
code2idx['GN9000'] = 30

def genre_cnt(x):
    counter = Counter(x)
    out = np.zeros(31)
    for gnr, cnt in counter.items():
        out[code2idx[gnr]] = cnt
    return out/len(x)

In [625]:
X_gn = pd.concat([X, pd.DataFrame(list(X['song_gn_gnr_basket_flatten'].apply(genre_cnt)))],axis=1)

In [626]:
X_gn = X_gn.add_prefix('gn_') 

In [627]:
X_gn.rename(columns = {'gn_id':'id','gn_plylst_title':'plylst_title','gn_song_gn_gnr_basket_flatten':'song_gn_gnr_basket_flatten','gn_artist_id_basket_flatten':'artist_id_basket_flatten',
                      'gn_artist_id_basket_count':'artist_id_basket_count','gn_song_gn_gnr_basket_count':'song_gn_gnr_basket_count','gn_tags':'tags','gn_year':'year','gn_month':'month',
                      'gn_season':'season','gn_year_section':'year_section'},inplace=True)

In [628]:
X_gn.head(3)

,id,plylst_title,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,tags,year,month,season,year_section,gn_0,gn_1,gn_2,gn_3,gn_4,gn_5,gn_6,gn_7,gn_8,gn_9,gn_10,gn_11,gn_12,gn_13,gn_14,gn_15,gn_16,gn_17,gn_18,gn_19,gn_20,gn_21,gn_22,gn_23,gn_24,gn_25,gn_26,gn_27,gn_28,gn_29,gn_30
0,104383.0,설레임 한가득 노래모음,"[GN2500, GN0100, GN0300, GN0100, GN0100, GN060...","[490981, 239366, 3305, 1582, 100391, 11538, 27...",55.0,10.0,"[설레임, 설렘, 사랑]",2020.0,2.0,4.0,5,0.27451,0.029412,0.147059,0.107843,0.107843,0.088235,0.0,0.098039,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.068627,0.0,0.009804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068627,0.0,0.0,0.0,0.0,0.0,0.0
1,28024.0,멜랑꼴리한 밤에 고독을 짓씹으며,"[GN0100, GN0900, GN0900, GN1000, GN1000, GN030...","[3305, 258383, 258383, 106732, 425140, 108356,...",23.0,11.0,"[우울, 새벽, 밤, 고독]",2018.0,1.0,4.0,4,0.27027,0.000000,0.189189,0.027027,0.027027,0.108108,0.0,0.027027,0.054054,0.216216,0.0,0.0,0.027027,0.027027,0.027027,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,120331.0,뜨거운 커피 한잔과 함께 듣는 노래,"[GN0500, GN0100, GN0500, GN0600, GN0500, GN060...","[681456, 242988, 242988, 770167, 103486, 24298...",16.0,7.0,"[커피, 잔잔한, 카페, 발라드]",2018.0,4.0,1.0,4,0.25000,0.027778,0.027778,0.000000,0.416667,0.194444,0.0,0.055556,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### tag 임배딩

In [620]:
from gensim.models import Word2Vec
tags = [p for p in X['tags'] if len(p) != 1]

m = Word2Vec(tags, size=16)

In [621]:
tag_vector = m.wv

tags = tag_vector.vocab.keys()
tag_vector_lst = [tag_vector[v] for v in tags]

In [629]:
def tag_embed(x):
    tem = []
    for tag in x:
        try:
            tem.append(tag_vector.get_vector(tag))
        except KeyError as e:
            pass
    if tem == []:
        return np.zeros(16)
    else:
        return np.mean(tem,axis=0)

In [653]:
X_total = pd.concat([X_gn ,pd.DataFrame(list(X['tags'].apply(tag_embed)))],axis=1)

In [655]:
X_total.rename(columns = {0:'tag_0',1:'tag_1',2:'tag_2',3:'tag_3',4:'tag_4',5:'tag_5',6:'tag_6',7:'tag_7',8:'tag_8',9:'tag_9',10:'tag_10',11:'tag_11',12:'tag_12',
                          13:'tag_13',14:'tag_14',15:'tag_15'},inplace=True)

In [656]:
X_total.head(3)

,id,plylst_title,song_gn_gnr_basket_flatten,artist_id_basket_flatten,artist_id_basket_count,song_gn_gnr_basket_count,tags,year,month,season,year_section,gn_0,gn_1,gn_2,gn_3,gn_4,gn_5,gn_6,gn_7,gn_8,gn_9,gn_10,gn_11,gn_12,gn_13,gn_14,gn_15,gn_16,gn_17,gn_18,gn_19,gn_20,gn_21,gn_22,gn_23,gn_24,gn_25,gn_26,gn_27,gn_28,gn_29,gn_30,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15
0,104383.0,설레임 한가득 노래모음,"[GN2500, GN0100, GN0300, GN0100, GN0100, GN060...","[490981, 239366, 3305, 1582, 100391, 11538, 27...",55.0,10.0,"[설레임, 설렘, 사랑]",2020.0,2.0,4.0,5,0.27451,0.029412,0.147059,0.107843,0.107843,0.088235,0.0,0.098039,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.068627,0.0,0.009804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068627,0.0,0.0,0.0,0.0,0.0,0.0,1.224304,1.683418,0.597946,1.564993,-1.088136,-2.614438,-0.955315,0.888589,2.248377,-0.799817,-0.970268,0.554964,0.464699,1.538041,0.052989,-0.054441
1,28024.0,멜랑꼴리한 밤에 고독을 짓씹으며,"[GN0100, GN0900, GN0900, GN1000, GN1000, GN030...","[3305, 258383, 258383, 106732, 425140, 108356,...",23.0,11.0,"[우울, 새벽, 밤, 고독]",2018.0,1.0,4.0,4,0.27027,0.000000,0.189189,0.027027,0.027027,0.108108,0.0,0.027027,0.054054,0.216216,0.0,0.0,0.027027,0.027027,0.027027,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.804882,-0.930250,0.160867,0.657510,-1.311092,-2.933626,-0.854970,-0.274359,0.663643,0.799360,0.880315,1.298008,1.778712,0.646767,2.571407,-0.221754
2,120331.0,뜨거운 커피 한잔과 함께 듣는 노래,"[GN0500, GN0100, GN0500, GN0600, GN0500, GN060...","[681456, 242988, 242988, 770167, 103486, 24298...",16.0,7.0,"[커피, 잔잔한, 카페, 발라드]",2018.0,4.0,1.0,4,0.25000,0.027778,0.027778,0.000000,0.416667,0.194444,0.0,0.055556,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.022725,0.522381,0.414624,1.473984,-0.372707,-2.786552,-0.396182,1.665532,1.368603,0.179974,0.399790,1.265499,1.673380,1.525942,2.518969,-0.762181


In [657]:
## 저장
## GN9000 - 장르분류X col 제거
X_total = X_total.drop('gn_30',axis=1)

X_total.to_json('X_total.json', orient='table')

In [658]:
## X_train : id 등등을 제거한 train set
X_total = X_total.drop('song_gn_gnr_basket_flatten',axis=1)
X_total = X_total.drop('artist_id_basket_flatten',axis=1)
X_total = X_total.drop('year', axis=1)
X_total = X_total.drop('month', axis=1)
X_total = X_total.drop('tags', axis=1)
X_train = X_total.drop('plylst_title',axis=1)
X_train = X_train.drop('id',axis=1)
X_train.head(3)

,artist_id_basket_count,song_gn_gnr_basket_count,season,year_section,gn_0,gn_1,gn_2,gn_3,gn_4,gn_5,gn_6,gn_7,gn_8,gn_9,gn_10,gn_11,gn_12,gn_13,gn_14,gn_15,gn_16,gn_17,gn_18,gn_19,gn_20,gn_21,gn_22,gn_23,gn_24,gn_25,gn_26,gn_27,gn_28,gn_29,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15
0,55.0,10.0,4.0,5,0.27451,0.029412,0.147059,0.107843,0.107843,0.088235,0.0,0.098039,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.068627,0.0,0.009804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068627,0.0,0.0,0.0,0.0,0.0,1.224304,1.683418,0.597946,1.564993,-1.088136,-2.614438,-0.955315,0.888589,2.248377,-0.799817,-0.970268,0.554964,0.464699,1.538041,0.052989,-0.054441
1,23.0,11.0,4.0,4,0.27027,0.000000,0.189189,0.027027,0.027027,0.108108,0.0,0.027027,0.054054,0.216216,0.0,0.0,0.027027,0.027027,0.027027,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.804882,-0.930250,0.160867,0.657510,-1.311092,-2.933626,-0.854970,-0.274359,0.663643,0.799360,0.880315,1.298008,1.778712,0.646767,2.571407,-0.221754
2,16.0,7.0,1.0,4,0.25000,0.027778,0.027778,0.000000,0.416667,0.194444,0.0,0.055556,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.022725,0.522381,0.414624,1.473984,-0.372707,-2.786552,-0.396182,1.665532,1.368603,0.179974,0.399790,1.265499,1.673380,1.525942,2.518969,-0.762181


In [659]:
## 저장 
X_train.to_json('X_train.json', orient='table')
"""
artist_cnt, gn_cnt, season, year_section -> 4차원
장르 cnt-hot-encoding : 비율 -> 30차원
태그 w2v -> 16차원

총 input features 50차원
"""

In [662]:
X_train = X_train.astype(float)
X_spr = spr.csr_matrix(X_train)
X_spr

<92056x50 sparse matrix of type '<class 'numpy.float64'>'
	with 2415613 stored elements in Compressed Sparse Row format>

In [663]:
target

<144674x92056 sparse matrix of type '<class 'numpy.float64'>'
	with 3626357 stored elements in Compressed Sparse Column format>

In [664]:
full = hstack((X_spr,target.T))
full = full.tocsc() ; full

<92056x144724 sparse matrix of type '<class 'numpy.float64'>'
	with 6041970 stored elements in Compressed Sparse Column format>

# Modeling

In [665]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
import random

## sample test

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
import joblib
from tqdm import tqdm

X = full[0:50].T.todense()
lst = list(range(92056))

for i in range(50,144724):
    song = full[i]
    y = song.data
    
    neg_1 = list(set(lst) - set(song.indices))

    k = round(len(y)*1.5)
    neg_idx = random.choices(neg_1, k=k)

    X_input = X[neg_idx+list(song.indices), :]

    Y = np.concatenate((y, np.zeros(k)))

    # X_neg = pd.DataFrame(np.array(X_neg).reshape(round(len(y)*1.5),50))
    # X_neg['y'] = 0

    # total = pd.concat([df_X_1,X_neg])

    # Y = df_X_1.pop('y')
    # X_input = df_X_1
    model_lgb = lgb.LGBMRegressor(n_estimators=300)

    model_lgb.fit(X_input, Y)

    lgb_train_pred = model_lgb.predict(X_input)

    print(rmsle(Y, lgb_train_pred))    
    # # model_xgb_2 = xgb.XGBRegressor(learning_rate=0.01,max_depth=3,n_estimators=300,objective ='reg:squarederror')
    # model_xgb_2.fit(X_input, Y)
    # xgb_train_pred = model_xgb_2.predict(X_input)

    #joblib.dump(model_lgb,'./XGBOOST/{}_model'.format(i-50))

    # print("{} rmse : {}".format(i-50,rmsle(Y, xgb_train_pred)))

# CF inference

In [ ]:
from tqdm import tqdm

def rec(pids):
  tt = 1

  res = []

  for pid in tqdm(pids): # pids로 test.index 들어옴. 따라서 플레이리스트마다 아래 과정 실행하는 것.
    p = np.zeros((n_songs,1)) 
    # n_songs) 플레이리스트에 있는 곡만 포함한듯?
    # song 개수만큼 0으로 채워진 array 생성
    p[test.loc[pid,'songs_id']] = 1 # 이번 플레이리스트에서 있는 song_id는 1로 바꿔줌

    val = train_songs_A.dot(p) 
    # train_songs_A > plylst-songs sparse 행렬 (115071 x 638336) / p > (638336 x 1)
    # 전체 플레이리스트 정보 x 이번 플레이리스트 > 둘 다 가지고 있는 곡 개수

    songs_already = test.loc[pid,"songs_id"]
    tags_already = test.loc[pid,"tags_id"]

    cand_song = train_songs_A_T.dot(val)
    # 플레이리스트 쌍마다 공통 곡의 수를 가중치로 하여 곡과 태그를 추천하는 방식
    # (638336 x 115071) x (115071 x 1) = (638336 x 1) > 어떤 플레이리스트에 있는 곡(값 1) x 겹치는 곡 수를 더해서 곡 별 일종의 점수 부여
    cand_song_idx = cand_song.reshape(-1).argsort()[-150:][::-1] # reshape) ex. (3,1) > (3,) / argsort) 작은 값부터 순서대로 데이터 인덱스 반환 / 큰 값부터 150개 자르고, 순서 뒤집음

    cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]
    rec_song_idx = [song_sid_id[i] for i in cand_song_idx]

    cand_tag = train_tags_A_T.dot(val)
    cand_tag_idx = cand_tag.reshape(-1).argsort()[-15:][::-1]

    cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
    rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]

    res.append({
        "id":plylst_nid_id[pid],
        "songs":rec_song_idx,
        "tags":rec_tag_idx
    })

    if tt % 1000 == 0:
      print(tt)

    tt += 1
  return res

In [ ]:
answers = rec(test.index)

  4%|▍         | 1003/23015 [00:50<17:52, 20.52it/s]

1000


  9%|▊         | 2004/23015 [01:40<17:32, 19.96it/s]

2000


 13%|█▎        | 3004/23015 [02:30<16:28, 20.24it/s]

...

 96%|█████████▌| 22004/23015 [18:08<00:48, 20.80it/s]

22000


100%|█████████▉| 23004/23015 [18:58<00:00, 19.87it/s]

23000


100%|██████████| 23015/23015 [18:58<00:00, 20.21it/s]


In [ ]:
write_json(answers,"results/results.json")

In [ ]:
evaluator = CustomEvaluator()
evaluator.evaluate("arena_data/answers/val.json", "arena_data/results/results.json")

# evaluator 주석처리한 에러 잡아줘야함 > 100곡, 10개 채우게
# 점수 준 게 막 엄청난 성능향상을 가져오진 않는듯

Music nDCG: 0.164403
Tag nDCG: 0.333865
Score: 0.189823
